In [0]:
import pandas as pd

In [0]:
#load data directly to spark dataframe. It will locate up to 128Mb data under single partition by default
loan_data = spark.read.format("csv") \
                      .option("inferSchema", True) \
                      .option("header", True) \
                      .option("sep", ",") \
                      .load("/FileStore/datasets/loan_data.csv")

In [0]:
# get number of partitions
loan_data.rdd.getNumPartitions()

Out[26]: 25

In [0]:
# to make data loaded to multiple partitions they have to be loaded to pandas dataframe first. Note that it won't work in local IDE (path)
loan_pandas_df = pd.read_csv("/dbfs/FileStore/datasets/loan_data.csv") #note that path is spelled differently

In [0]:
# next create spark dataframe from pandas dataframe
loan_df = spark.createDataFrame(loan_pandas_df)

In [0]:
# check number of partitions. Now it is 4 (default == number of cores in selected spark cluster)
loan_df.rdd.getNumPartitions()

Out[29]: 4

In [0]:
# to check the default number of partitions for current cluster
spark.sparkContext.defaultParallelism

Out[30]: 4

In [0]:
# get data from all partitions into a single (multidimentional) list
loan_df.rdd.glom().collect()

Out[31]: [[Row(ID=1, Default=0, Loan_type='Car ', Gender='Female', Age=30, Degree='HS ', Income=114885, Credit_score=641, Loan_length=0, Signers=2, Citizenship='Citizen '),
 Row(ID=2, Default=0, Loan_type='Home', Gender='Female', Age=43, Degree='HS ', Income=95770, Credit_score=534, Loan_length=7, Signers=2, Citizenship='Citizen '),
 Row(ID=3, Default=0, Loan_type='Home', Gender='Male ', Age=39, Degree='HS ', Income=94220, Credit_score=558, Loan_length=5, Signers=2, Citizenship='Citizen '),
 Row(ID=4, Default=0, Loan_type='Home', Gender='Male ', Age=39, Degree='College ', Income=58946, Credit_score=622, Loan_length=7, Signers=2, Citizenship='Citizen '),
 Row(ID=5, Default=0, Loan_type='Car ', Gender='Female', Age=42, Degree='HS ', Income=79754, Credit_score=702, Loan_length=2, Signers=1, Citizenship='Citizen '),
 Row(ID=6, Default=0, Loan_type='Car ', Gender='Female', Age=57, Degree='Graduate', Income=119627, Credit_score=624, Loan_length=3, Signers=2, Citizenship='Non-citizen'),
 Row(ID=7, Default=0, Loan_type='Car ', Gender='Male ', Age=41, Degree='HS ', Income=78765, Credit_score=594, Loan_length=3, Signers=2, Citizenship='Citizen '),
 Row(ID=8, Default=0, Loan_type='Car ', Gender='Female', Age=53, Degree='HS ', Income=123167, Credit_score=652, Loan_length=2, Signers=2, Citizenship='Non-citizen'),
 Row(ID=9, Default=0, Loan_type='Car ', Gender='Female', Age=44, Degree='HS ', Income=116175, Credit_score=694, Loan_length=3, Signers=2, Citizenship='Citizen '),
 Row(ID=10, Default=0, Loan_type='Car ', Gender='Female', Age=57, Degree='College ', Income=111085, Credit_score=586, Loan_length=3, Signers=2, Citizenship='Citizen '),
 Row(ID=11, Default=0, Loan_type='Car ', Gender='Male ', Age=50, Degree='HS ', Income=54387, Credit_score=593, Loan_length=2, Signers=2, Citizenship='Citizen '),
 Row(ID=12, Default=0, Loan_type='Car ', Gender='Male ', Age=39, Degree='HS ', Income=89468, Credit_score=600, Loan_length=0, Signers=2, Citizenship='Citizen '),
 Row(ID=13, Default=0, Loan_type='Car ', Gender='Male ', Age=52, Degree='HS ', Income=118613, Credit_score=622, Loan_length=0, Signers=2, Citizenship='Citizen '),
 Row(ID=14, Default=0, Loan_type='Car ', Gender='Male ', Age=37, Degree='HS ', Income=84429, Credit_score=615, Loan_length=0, Signers=2, Citizenship='Citizen '),
 Row(ID=15, Default=0, Loan_type='Car ', Gender='Female', Age=46, Degree='College ', Income=113281, Credit_score=577, Loan_length=3, Signers=2, Citizenship='Citizen '),
 Row(ID=16, Default=0, Loan_type='Home', Gender='Female', Age=45, Degree='College ', Income=93253, Credit_score=580, Loan_length=10, Signers=1, Citizenship='Citizen '),
 Row(ID=17, Default=0, Loan_type='Car ', Gender='Male ', Age=45, Degree='College ', Income=89304, Credit_score=665, Loan_length=3, Signers=1, Citizenship='Citizen '),
 Row(ID=18, Default=0, Loan_type='Car ', Gender='Male ', Age=45, Degree='College ', Income=126260, Credit_score=638, Loan_length=1, Signers=2, Citizenship='Citizen '),
 Row(ID=19, Default=0, Loan_type='Car ', Gender='Male ', Age=42, Degree='HS ', Income=77616, Credit_score=655, Loan_length=3, Signers=2, Citizenship='Citizen '),
 Row(ID=20, Default=0, Loan_type='Car ', Gender='Female', Age=34, Degree='HS ', Income=88568, Credit_score=630, Loan_length=0, Signers=2, Citizenship='Citizen '),
 Row(ID=21, Default=0, Loan_type='Car ', Gender='Male ', Age=47, Degree='HS ', Income=118155, Credit_score=730, Loan_length=3, Signers=1, Citizenship='Citizen '),
 Row(ID=22, Default=0, Loan_type='Car ', Gender='Female', Age=50, Degree='College ', Income=132630, Credit_score=625, Loan_length=3, Signers=2, Citizenship='Citizen '),
 Row(ID=23, Default=0, Loan_type='Home', Gender='Male ', Age=46, Degree='HS ', Income=99880, Credit_score=551, Loan_length=8, Signers=2, Citizenship='Citizen '),
 Row(ID=24, Default=1, Loan_type='Home', Gender='Female', Age=53, Degree='Graduate', Income=118848, Credit_score=701, Loan_length=8, Signers=1, Citizenship='Citizen '),
 Row(ID=25, Default=1, Loan_type='Ho

In [0]:
type(loan_df.rdd.glom().collect())

Out[32]: list

In [0]:
# if you check the length of the outer list, it is 4 (contains 4 inner lists each for partition) 
len(loan_df.rdd.glom().collect())

Out[33]: 4

In [0]:
# let's move back to the original spark dataframe, loaded to single partition
loan_data.filter(loan_data['Degree'] == 'Graduate').display() # task creates 1 job run in 1 partition

ID,Default,Loan_type,Gender,Age,Degree,Income,Credit_score,Loan_length,Signers,Citizenship
6,0,Car,Female,57,Graduate,119627,624,3,2,Non-citizen
24,1,Home,Female,53,Graduate,118848,701,8,1,Citizen
27,0,Car,Female,44,Graduate,122276,709,3,1,Non-citizen
32,0,Car,Female,55,Graduate,120783,585,3,1,Citizen
80,0,Car,Female,46,Graduate,87658,688,2,2,Citizen
89,0,Car,Female,42,Graduate,118537,623,2,2,Citizen
105,0,Car,Male,54,Graduate,118278,471,3,2,Citizen
125,0,Car,Male,52,Graduate,101998,636,3,2,Non-citizen
144,0,Home,Male,38,Graduate,107839,682,12,2,Citizen
190,0,Car,Male,49,Graduate,126511,716,3,2,Citizen


In [0]:
# now perform the same task on spark df created from pandas dataframe (has 4 partitions)
loan_df.filter(loan_df['Degree'] == 'Graduate').display() #here 1st job with 1 task/1 partition, and 2nd job with 3 tasks/3 partitions

ID,Default,Loan_type,Gender,Age,Degree,Income,Credit_score,Loan_length,Signers,Citizenship
6,0,Car,Female,57,Graduate,119627,624,3,2,Non-citizen
24,1,Home,Female,53,Graduate,118848,701,8,1,Citizen
27,0,Car,Female,44,Graduate,122276,709,3,1,Non-citizen
32,0,Car,Female,55,Graduate,120783,585,3,1,Citizen
80,0,Car,Female,46,Graduate,87658,688,2,2,Citizen
89,0,Car,Female,42,Graduate,118537,623,2,2,Citizen
105,0,Car,Male,54,Graduate,118278,471,3,2,Citizen
125,0,Car,Male,52,Graduate,101998,636,3,2,Non-citizen
144,0,Home,Male,38,Graduate,107839,682,12,2,Citizen
190,0,Car,Male,49,Graduate,126511,716,3,2,Citizen


In [0]:
# get max size per partition in bytes (appr. 134Mb)
spark.conf.get("spark.sql.files.maxPartitionBytes")

Out[36]: '10000'

In [0]:
# set max partition size manually
spark.conf.set("spark.sql.files.maxPartitionBytes", 10000)

In [0]:
# now recreate spark dataframe and check the number of partitions
loan_data_with_partitions = spark.read.format("csv") \
                      .option("inferSchema", True) \
                      .option("header", True) \
                      .option("sep", ",") \
                      .load("/FileStore/datasets/loan_data.csv")
loan_data_with_partitions.rdd.getNumPartitions() #returns 25 partitions

Out[39]: 25

In [0]:
# now the same command invoke 3 Jobs (1+4+20=25 tasks, each per partition)
loan_data_with_partitions.filter(loan_data_with_partitions['Degree'] =='Graduate').display()

ID,Default,Loan_type,Gender,Age,Degree,Income,Credit_score,Loan_length,Signers,Citizenship
6,0,Car,Female,57,Graduate,119627,624,3,2,Non-citizen
24,1,Home,Female,53,Graduate,118848,701,8,1,Citizen
27,0,Car,Female,44,Graduate,122276,709,3,1,Non-citizen
32,0,Car,Female,55,Graduate,120783,585,3,1,Citizen
80,0,Car,Female,46,Graduate,87658,688,2,2,Citizen
89,0,Car,Female,42,Graduate,118537,623,2,2,Citizen
105,0,Car,Male,54,Graduate,118278,471,3,2,Citizen
125,0,Car,Male,52,Graduate,101998,636,3,2,Non-citizen
144,0,Home,Male,38,Graduate,107839,682,12,2,Citizen
190,0,Car,Male,49,Graduate,126511,716,3,2,Citizen


In [0]:
# repartition data
#currently it is 4
loan_df.rdd.getNumPartitions()

Out[41]: 4

In [0]:
# perform repartition. Now it is 8
repartitioned_loan_df = loan_df.repartition(8)
repartitioned_loan_df.rdd.getNumPartitions()

Out[42]: 8

In [0]:
# another way to get the number of partitions
len(repartitioned_loan_df.rdd.glom().collect())

Out[43]: 8

In [0]:
# to perform manual partitioning based on column value, first we have to turn off adaptive partitioning
# this command returns 200, despite just 3 values in target column. Because shuffling is performed by default

spark.conf.set("spark.sql.adaptive.enabled", False)

repartitioned_col_loan_df =loan_df.repartition('Degree')
repartitioned_col_loan_df.rdd.getNumPartitions()

Out[44]: 200

In [0]:
spark.conf.get("spark.sql.shuffle.partitions")

Out[45]: '200'

In [0]:
# command reveals that all of partitions are empty, but 3 partitions containing data
repartitioned_col_loan_df.rdd.glom().collect()

Out[47]: [[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [Row(ID=1, Default=0, Loan_type='Car ', Gender='Female', Age=30, Degree='HS ', Income=114885, Credit_score=641, Loan_length=0, Signers=2, Citizenship='Citizen '),
 Row(ID=2, Default=0, Loan_type='Home', Gender='Female', Age=43, Degree='HS ', Income=95770, Credit_score=534, Loan_length=7, Signers=2, Citizenship='Citizen '),
 Row(ID=3, Default=0, Loan_type='Home', Gender='Male ', Age=39, Degree='HS ', Income=94220, Credit_score=558, Loan_length=5, Signers=2, Citizenship='Citizen '),
 Row(ID=5, Default=0, Loan_type='Car ', Gender='Female', Age=42, Degree='HS ', Income=79754, Credit_score=702, Loan_length=2, Signers=1, Citizenship='Citizen '),
 Row(ID=7, Default=0, Loan_type='Car ', Gender='Male ', Age=41, Degree='HS ', Income=78765, Credit_score=594, Loan_length=3, Signers=2, Citizenship='Citizen '),
 Row(ID=8, Default=0, Loan_type='Car ', Gender='Female', Age=53, Degree='HS ', Income=123167, Credit_score=652, Loan_length=2, Signers=2, Citizenship='Non-citizen'),
 Row(ID=9, Default=0, Loan_type='Car ', Gender='Female', Age=44, Degree='HS ', Income=116175, Credit_score=694, Loan_length=3, Signers=2, Citizenship='Citizen '),
 Row(ID=11, Default=0, Loan_type='Car ', Gender='Male ', Age=50, Degree='HS ', Income=54387, Credit_score=593, Loan_length=2, Signers=2, Citizenship='Citizen '),
 Row(ID=12, Default=0, Loan_type='Car ', Gender='Male ', Age=39, Degree='HS ', Income=89468, Credit_score=600, Loan_length=0, Signers=2, Citizenship='Citizen '),
 Row(ID=13, Default=0, Loan_type='Car ', Gender='Male ', Age=52, Degree='HS ', Income=118613, Credit_score=622, Loan_length=0, Signers=2, Citizenship='Citizen '),
 Row(ID=14, Default=0, Loan_type='Car ', Gender='Male ', Age=37, Degree='HS ', Income=84429, Credit_score=615, Loan_length=0, Signers=2, Citizenship='Citizen '),
 Row(ID=19, Default=0, Loan_type='Car ', Gender='Male ', Age=42, Degree='HS ', Income=77616, Credit_score=655, Loan_length=3, Signers=2, Citizenship='Citizen '),
 Row(ID=20, Default=0, Loan_type='Car ', Gender='Female', Age=34, Degree='HS ', Income=88568, Credit_score=630, Loan_length=0, Signers=2, Citizenship='Citizen '),
 Row(ID=21, Default=0, Loan_type='Car ', Gender='Male ', Age=47, Degree='HS ', Income=118155, Credit_score=730, Loan_length=3, Signers=1, Citizenship='Citizen '),
 Row(ID=23, Default=0, Loan_type='Home', Gender='Male ', Age=46, Degree='HS ', Income=99880, Credit_score=551, Loan_length=8, Signers=2, Citizenship='Citizen '),
 Row(ID=26, Default=0, Loan_type='Car ', Gender='Female', Age=41, Degree='HS ', Income=90653, Credit_score=641, Loan_length=0, Signers=2, Citizenship='Citizen '),
 Row(ID=31, Default=0, Loan_type='Car ', Gender='Male ', Age=46, Degree='HS ', Income=90207, Credit_score=663, Loan_length=3, Signers=2, Citizenship='Citizen '),
 Row(ID=33, Default=0, Loan_type='Home', Gender='Male ', Age=41, Degree='HS ', Income=104234, Credit_score=586, Loan_length=7, Signers=2, Citizenship='Citizen '),
 Row(ID=36, Default=0, Loan_type='Car ', Gender='Male ', Age=41, Degree='HS ', Income=114009, Credit_score=780, Loan_length=3, Signers=2, Citizenship='Citizen '),
 Row(ID=38, Default=0, Loan_type='Car ', Gender='Male ', Age=44, Degree='HS ', Income=93651, Credit_score=639, Loan_length=1, Signers=2, Citizenship='Citizen '),
 Row(ID=39, Default=1, Loan_type='Home', Gender='Male ', Age=36, Degree='HS ', Income=90710, Credit_score=667, Loan_length=5, Signers=2, Citizenship='Citizen '),
 Row(ID=40, Default=0, Loan_type='Home', Gender='Female', Age=43, Degree='HS ', Income=75956, Credit_score=592, Loan_length=6, Signers=2, Citizenship='Citizen '),
 Row(ID=41, Default=0, Loan_type='Home', Gender='Male ', Age=41, Degree='HS ', Income=80480, Credit_score=689, Loan_length=5, Signers=2, Citizenship='Citizen '),
 Row(ID=42, Default=0, Loan_type='Home', Gender='Male ', Age=50, Degree='HS ', Income=113200, Credit_score=573, Loan_length=6, Signers=2, Citizenship='Citizen '),
 Row(ID=43, Default=

In [0]:
# decrease number of partitions to 2
coalesced_loan_df = loan_df.coalesce(2)
coalesced_loan_df.rdd.getNumPartitions()

Out[49]: 2

In [0]:
# coalesce() doesn't work for increasing partitions (returns the same number instead)
coalesced_loan_df = loan_df.coalesce(8)
coalesced_loan_df.rdd.getNumPartitions()

Out[50]: 4